In [63]:
import pandas as pd
from functools import reduce
from pathlib import Path
import os
import cx_Oracle
import numpy as np
import tarfile
import subprocess

# Questionaire data

In [ ]:
code_dir = Path(os.getcwd())

# directory where the data is stored
base_path = code_dir.parent / "data"

# File 2 CBCL data
file2_path = f"{base_path}/mh_p_cbcl.csv"

file2 = pd.read_csv(file2_path)

file2_baseline_filtered = file2[file2['eventname'] == 'baseline_year_1_arm_1']

#drop score
file2_drop = file2_baseline_filtered.loc[:, ~file2_baseline_filtered.columns.str.endswith(('_r', '_t', '_m', '_nm', '_nm_2', '___1'))]
#drop eventname
file2_drop = file2_drop.drop(columns=['eventname'])
file2_final_cleaned = file2_drop.dropna(axis=0)

C:\Users\Makka Papa\AppData\Local\Temp\ipykernel_3008\2214807639.py:14: DtypeWarning: Columns (124,128,132,136,140,144,148,152,156,160,164,168,172,176,180,184,188,192,196,200) have mixed types. Specify dtype option on import or set low_memory=False.
  file2 = pd.read_csv(file2_path)


In [7]:
file2_final_cleaned.to_csv(f'{base_path}/cbcl_data_remove_useless_items.csv')


# Neuroimaging data

In [ ]:
# set the path of Oracle Instant Client
os.environ["PATH"] = "C:\\oracle\\instantclient_23_7;" + os.environ["PATH"]
print(cx_Oracle.clientversion())

"""
Username: k21116947_1237171

Host: mindarvpc.cqahbwk3l1mb.us-east-1.rds.amazonaws.com

Port: 1521

Service Name: ORCL
"""


dsn = cx_Oracle.makedsn(
    "mindarvpc.cqahbwk3l1mb.us-east-1.rds.amazonaws.com", 1521, service_name="ORCL"
)
conn = cx_Oracle.connect(user="k21116947_1237171", password="KVJ3a4UdWrczt3", dsn=dsn)

cursor = conn.cursor()

query ="""
SELECT ENDPOINT
FROM S3_LINKS
WHERE ENDPOINT LIKE '%baseline%' AND (ENDPOINT LIKE '%rsfMRI%' OR ENDPOINT LIKE '%T1%') AND ENDPOINT LIKE '%MPROC%' 
--AND ENDPOINT LIKE '%NDARINV05ATJ1V1%'
"""

cursor.execute(query)

s3_samples = [row[0] for row in cursor.fetchall()]

cursor.close()
conn.close()

for url in s3_samples:
   print(url)

# np.savetxt("../data/s3_links.txt", s3_samples, fmt="%s")


In [68]:
# download data
def download_data():
    command = "downloadcmd -dp 1237171 -t ../data/s3_links.txt -d ../notebooks"

    result = subprocess.run(command, shell=True)

    if result.returncode == 0:
        print("succeed")
    else:
        print("failed")
download_data()

succeed


In [8]:
def s3lnik_table(which_file='rsfMRI'):
    """
    Extracts NDAR subject_id from a specified column and inserts it as the first column.

    Parameters:
        path_col (str): The name of the column containing S3 paths (default: 's3_path').
        which_file (str): The file type keyword to filter paths (e.g., 'rsfMRI' or 'T1').

    Returns:
        DataFrame: A new DataFrame where 'subject_id' is the first column.
    """
    df = pd.DataFrame({
        's3_path_'+ which_file: [url for url in s3_samples if '-' + which_file + '_' in url]
    })
    df = df.copy()
    df['subject_id'] = df['s3_path_'+ which_file].str.extract(r'/(NDARINV[A-Z0-9]+)_')
    cols = ['subject_id'] + [col for col in df.columns if col != 'subject_id']
    return df[cols]


# np.savetxt("../data/s3_link.txt", df['s3_path'].values, fmt="%s")

In [41]:
rsfmri_expanded = (
    s3lnik_table('rsfMRI').dropna(subset=['s3_path_rsfMRI'])  # drop rows with missing s3_path
      .groupby('subject_id')['s3_path_rsfMRI']  # group by subject_id
      .apply(lambda x: pd.Series(x.values))  # expand to multiple columns
      .unstack()  # row to column
)
rsfmri_expanded.columns = [f's3_path_rsfMRI{i+1}' for i in range(rsfmri_expanded.shape[1])]

s3link = pd.merge(s3lnik_table('T1'), rsfmri_expanded, on='subject_id')


s3link
# s3link.to_csv(f'../data/s3link.csv', index=False)

,subject_id,s3_path_T1,s3_path_rsfMRI1,s3_path_rsfMRI2,s3_path_rsfMRI3,s3_path_rsfMRI4,s3_path_rsfMRI5,s3_path_rsfMRI6
0,NDARINVMXMAHBU0,s3://NDAR_Central_3/submission_52682/abcd-mpro...,s3://NDAR_Central_3/submission_52682/abcd-mpro...,s3://NDAR_Central_3/submission_52682/abcd-mpro...,s3://NDAR_Central_3/submission_52682/abcd-mpro...,s3://NDAR_Central_3/submission_52682/abcd-mpro...,NaN,NaN
1,NDARINV5VJHAFVK,s3://NDAR_Central_3/submission_52682/abcd-mpro...,s3://NDAR_Central_3/submission_52682/abcd-mpro...,s3://NDAR_Central_3/submission_52682/abcd-mpro...,s3://NDAR_Central_3/submission_52682/abcd-mpro...,s3://NDAR_Central_3/submission_52682/abcd-mpro...,NaN,NaN
2,NDARINV7XF3TWTY,s3://NDAR_Central_3/submission_52682/abcd-mpro...,s3://NDAR_Central_3/submission_52682/abcd-mpro...,s3://NDAR_Central_3/submission_52682/abcd-mpro...,s3://NDAR_Central_3/submission_52682/abcd-mpro...,s3://NDAR_Central_3/submission_52682/abcd-mpro...,NaN,NaN
3,NDARINV8M3JENZ9,s3://NDAR_Central_3/submission_52682/abcd-mpro...,s3://NDAR_Central_3/submission_52682/abcd-mpro...,s3://NDAR_Central_3/submission_52682/abcd-mpro...,s3://NDAR_Central_3/submission_52682/abcd-mpro...,s3://NDAR_Central_3/submission_52682/abcd-mpro...,NaN,NaN
4,NDARINV2LL2Y78J,s3://NDAR_Central_3/submission_52682/abcd-mpro...,s3://NDAR_Central_3/submission_52682/abcd-mpro...,s3://NDAR_Central_3/submission_52682/abcd-mpro...,s3://NDAR_Central_3/submission_52682/abcd-mpro...,s3://NDAR_Central_3/submission_52682/abcd-mpro...,s3://NDAR_Central_3/submission_52682/abcd-mpro...,NaN
...,...,...,...,...,...,...,...,...
5863,NDARINVFC8MZRJA,s3://NDAR_Central_3/submission_52682/abcd-mpro...,s3://NDAR_Central_3/submission_52682/abcd-mpro...,s3://NDAR_Central_3/submission_52682/abcd-mpro...,s3://NDAR_Central_3/submission_52682/abcd-mpro...,s3://NDAR_Central_3/submission_52682/abcd-mpro...,NaN,NaN
5864,NDARINVPNF8BAGH,s3://NDAR_Central_3/submission_52682/abcd-mpro...,s3://NDAR_Central_3/submission_52682/abcd-mpro...,s3://NDAR_Central_3/submission_52682/abcd-mpro...,s3://NDAR_Central_3/submission_52682/abcd-mpro...,s3://NDAR_Central_3/submission_52682/abcd-mpro...,NaN,NaN
5865,NDARINVUG6DTWVU,s3://NDAR_Central_3/submission_52682/abcd-mpro...,s3://NDAR_Central_3/submission_52682/abcd-mpro...,s3://NDAR_Central_3/submission_52682/abcd-mpro...,s3://NDAR_Central_3/submission_52682/abcd-mpro...,NaN,NaN,NaN
5866,NDARINVUE7CU1V9,s3://NDAR_Central_3/submission_52682/abcd-mpro...,s3://NDAR_Central_3/submission_52682/abcd-mpro...,s3://NDAR_Central_3/submission_52682/abcd-mpro...,s3://NDAR_Central_3/submission_52682/abcd-mpro...,s3://NDAR_Central_3/submission_52682/abcd-mpro...,NaN,NaN


In [ ]:
def extract_tgz_and_delete(tgz_path, extract_path="."):
    # 解压 .tgz 文件
    with tarfile.open(tgz_path, "r:gz") as tar:
        tar.extractall(path=extract_path)
        print(f"unzip done: {tgz_path} → {extract_path}")

    # 删除原始 .tgz 文件
    os.remove(tgz_path)
    print(f".tgz files deleted: {tgz_path}")

"""
Example:
extract_tgz_and_delete("example.tgz", "output_folder")
"""

# FC matrix

In [42]:
data = pd.read_csv(r"G:/ABCD/data/mri_y_rsfmr_cor_gp_gp.csv")
data = data[data['eventname'] == 'baseline_year_1_arm_1'].drop(columns=['eventname']).dropna()

data

,src_subject_id,rsfmri_c_ngd_ad_ngd_ad,rsfmri_c_ngd_ad_ngd_cgc,rsfmri_c_ngd_ad_ngd_ca,rsfmri_c_ngd_ad_ngd_dt,rsfmri_c_ngd_ad_ngd_dla,rsfmri_c_ngd_ad_ngd_fo,rsfmri_c_ngd_ad_ngd_n,rsfmri_c_ngd_ad_ngd_rspltp,rsfmri_c_ngd_ad_ngd_sa,...,rsfmri_c_ngd_vs_ngd_dt,rsfmri_c_ngd_vs_ngd_dla,rsfmri_c_ngd_vs_ngd_fo,rsfmri_c_ngd_vs_ngd_n,rsfmri_c_ngd_vs_ngd_rspltp,rsfmri_c_ngd_vs_ngd_sa,rsfmri_c_ngd_vs_ngd_smh,rsfmri_c_ngd_vs_ngd_smm,rsfmri_c_ngd_vs_ngd_vta,rsfmri_c_ngd_vs_ngd_vs
0,NDAR_INV003RTV85,0.471330,0.256267,-0.076960,-0.116451,0.022202,-0.036302,-0.057183,-0.048132,0.006416,...,-0.088894,0.055355,-0.076806,-0.006004,0.228366,-0.150867,-0.064416,0.098679,-0.155134,0.371960
1,NDAR_INV005V6D2C,0.279435,0.116256,0.063664,-0.024781,-0.000840,-0.023421,-0.016284,0.022696,0.056241,...,-0.136596,0.017415,-0.111623,-0.077958,0.049159,-0.065420,-0.061459,0.075145,-0.071943,0.346746
2,NDAR_INV007W6H7B,0.294463,0.209772,-0.071834,-0.138693,-0.035168,0.044412,-0.041321,-0.185939,0.071985,...,-0.136426,-0.018266,-0.096822,-0.050623,0.002644,-0.017955,-0.060259,0.070566,-0.076156,0.400397
3,NDAR_INV00BD7VDC,0.241918,0.163942,-0.090651,-0.044039,0.012523,-0.022455,-0.005482,-0.124627,0.095889,...,-0.117847,0.007997,-0.117152,-0.028143,0.079898,-0.066724,-0.029856,-0.038823,-0.092095,0.334313
6,NDAR_INV00CY2MDM,0.395300,0.180940,-0.142808,-0.073424,-0.049414,-0.144481,-0.024881,-0.012894,-0.036592,...,-0.202610,0.014283,-0.200163,-0.091514,0.154572,-0.210839,0.033060,0.159728,-0.125210,0.485745
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22117,NDAR_INVZZLZCKAY,0.315367,0.176642,-0.010926,-0.019208,-0.030765,-0.062264,0.021828,-0.043990,0.140031,...,-0.075473,-0.008800,-0.103751,-0.035379,0.183831,-0.019576,-0.045539,-0.012037,-0.054182,0.333367
22120,NDAR_INVZZNX6W2P,0.343187,0.142774,-0.055901,-0.046064,-0.006279,0.013171,0.017289,-0.064605,-0.014693,...,-0.164360,0.059505,-0.157277,-0.037063,0.264306,-0.143545,-0.082002,0.015764,-0.126353,0.484629
22123,NDAR_INVZZPKBDAC,0.404070,0.300714,-0.167735,-0.093816,-0.044867,0.019305,0.049103,-0.157262,0.192685,...,-0.183933,0.022202,-0.130174,-0.066710,0.179635,-0.012360,-0.098269,0.069790,-0.126898,0.462496
22125,NDAR_INVZZZ2ALR6,0.411105,0.209193,-0.041473,-0.063504,-0.102791,-0.046049,-0.009828,-0.162120,0.026045,...,-0.152191,0.022720,-0.132211,-0.046361,0.170122,-0.094498,0.000854,0.030236,-0.153170,0.351791


In [ ]:
import pandas as pd
from scipy.stats import pearsonr
import itertools

df = data.iloc[:, 1:] 
# 假设你的 DataFrame 是 df
columns = df.columns
results = []

# 遍历所有列的组合（两两配对）
for col1, col2 in itertools.combinations(columns, 2):
    try:
        corr, pval = pearsonr(df[col1], df[col2])
        results.append({
            'col1': col1,
            'col2': col2,
            'correlation': corr,
            'p_value': pval
        })
    except Exception as e:
        print(f"跳过 {col1}, {col2}: {e}")

# 转换为 DataFrame
result_df = pd.DataFrame(results)

# 筛选 p < 0.05 的高度相关列对
significant = result_df[result_df['p_value'] < 0.02].sort_values('p_value')

In [58]:
strong_corr = significant[
    (significant['correlation'].abs() == 1.0)
]

strong_corr

,col1,col2,correlation,p_value
5368,rsfmri_c_ngd_ca_ngd_smh,rsfmri_c_ngd_smh_ngd_ca,1.0,0.0
5075,rsfmri_c_ngd_ca_ngd_rspltp,rsfmri_c_ngd_rspltp_ngd_ca,1.0,0.0
4927,rsfmri_c_ngd_ca_ngd_n,rsfmri_c_ngd_n_ngd_ca,1.0,0.0
4477,rsfmri_c_ngd_ca_ngd_dt,rsfmri_c_ngd_dt_ngd_ca,1.0,0.0
5513,rsfmri_c_ngd_ca_ngd_smm,rsfmri_c_ngd_smm_ngd_ca,1.0,0.0
5800,rsfmri_c_ngd_ca_ngd_vs,rsfmri_c_ngd_vs_ngd_ca,1.0,0.0
5657,rsfmri_c_ngd_ca_ngd_vta,rsfmri_c_ngd_vta_ngd_ca,1.0,0.0
3401,rsfmri_c_ngd_cgc_ngd_sa,rsfmri_c_ngd_sa_ngd_cgc,1.0,0.0
3560,rsfmri_c_ngd_cgc_ngd_smh,rsfmri_c_ngd_smh_ngd_cgc,1.0,0.0
3875,rsfmri_c_ngd_cgc_ngd_vta,rsfmri_c_ngd_vta_ngd_cgc,1.0,0.0


In [56]:
# 保留绝对值大于 0.8 的相关性（正相关或负相关）
strong_corr = significant[
    (significant['correlation'].abs() > 0.6) &
    (significant['correlation'].abs() < 0.9)
]

strong_corr

,col1,col2,correlation,p_value
0,rsfmri_c_ngd_ad_ngd_ad,rsfmri_c_ngd_ad_ngd_cgc,0.613490,0.0
5942,rsfmri_c_ngd_dt_ngd_cgc,rsfmri_c_ngd_dt_ngd_dla,0.624497,0.0
3874,rsfmri_c_ngd_cgc_ngd_vta,rsfmri_c_ngd_vta_ngd_ad,0.665815,0.0
5954,rsfmri_c_ngd_dt_ngd_cgc,rsfmri_c_ngd_dla_ngd_dt,0.624497,0.0
6686,rsfmri_c_ngd_dt_ngd_n,rsfmri_c_ngd_vs_ngd_n,-0.667475,0.0
6614,rsfmri_c_ngd_dt_ngd_n,rsfmri_c_ngd_n_ngd_vs,-0.667475,0.0
6207,rsfmri_c_ngd_dt_ngd_dt,rsfmri_c_ngd_dla_ngd_dt,-0.672338,0.0
6203,rsfmri_c_ngd_dt_ngd_dt,rsfmri_c_ngd_dt_ngd_vs,-0.641105,0.0
6197,rsfmri_c_ngd_dt_ngd_dt,rsfmri_c_ngd_dt_ngd_n,0.636310,0.0
6195,rsfmri_c_ngd_dt_ngd_dt,rsfmri_c_ngd_dt_ngd_dla,-0.672338,0.0
